In [ ]:
"""
Tutorial for the Million Song Dataset

by Thierry Bertin-Mahieux (2011) Columbia University
   tb2332@columbia.edu
   Copyright 2011 T. Bertin-Mahieux, All Rights Reserved

This tutorial will walk you through a quick experiment
using the Million Song Dataset (MSD). We will actually be working
on the 10K songs subset for speed issues, but the code should
transpose seamlessly.

In this tutorial, we do simple metadata analysis. We look at
which artist has the most songs by iterating over the whole
dataset and using an SQLite database.

You need to have the MSD code downloaded from GITHUB.
See the MSD website for details:
http://labrosa.ee.columbia.edu/millionsong/

If you have any questions regarding the dataset or this tutorial,
please first take a look at the website. Send us an email
if you haven't found the answer.

Note: this tutorial is developed using Python 2.6
      on an Ubuntu machine. PDF created using 'pyreport'.
"""

# usual imports
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np # get it at: http://numpy.scipy.org/
# path to the Million Song Dataset subset (uncompressed)
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_subset_path='data/MillionSongSubset/'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' # sanity check
# path to the Million Song Dataset code
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_code_path='data/MSongsDB-master'
assert os.path.isdir(msd_code_path),'wrong path' # sanity check
# we add some paths to python so we can import MSD code
# Ubuntu: you can change the environment variable PYTHONPATH
# in your .bashrc file so you do not have to type these lines
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

# imports specific to the MSD
import hdf5_getters as GETTERS

# the following function simply gives us a nice string for
# a time lag in seconds
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

# we define this very useful function to iterate the files
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

# we can now easily count the number of files in the dataset
print 'number of song files:',apply_to_all_files(msd_subset_data_path)

In [ ]:
# let's now get all artist names in a set(). One nice property:
# if we enter many times the same artist, only one will be kept.
all_artist_names = set()

# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add( artist_name )
    h5.close()
    
# let's apply the previous function to all files
# we'll also measure how long it takes
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_artist_name)
t2 = time.time()
print 'all artist names extracted in:',strtimedelta(t1,t2)

In [99]:
# let's now get all energies in a list. One nice property:
# if we enter many times the same artist, only one will be kept.
all_energies = list()

# we define the function to apply to all files
def func_to_get_energy(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    energy = GETTERS.get_energy(h5)
    all_energies.append(energy)
    h5.close()
    
# let's apply the previous function to all files
# we'll also measure how long it takes
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_energy)
t2 = time.time()
print 'all energies extracted in:',strtimedelta(t1,t2)

all energies extracted in: 0:06:01.161484


In [100]:
sum(all_energies)

0.0

In [ ]:
# let's see some of the content of 'all_artist_names'
print 'found',len(all_artist_names),'unique artist names'
for k in range(5):
    print list(all_artist_names)[k]
    

In [ ]:
# this is too long, and the work of listing artist names has already
# been done. Let's redo the same task using an SQLite database.
# We connect to the provided database: track_metadata.db
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
# we build the SQL query
q = "SELECT DISTINCT artist_name FROM songs"
# we query the database
t1 = time.time()
res = conn.execute(q)
all_artist_names_sqlite = res.fetchall()
t2 = time.time()
print 'all artist names extracted (SQLite) in:',strtimedelta(t1,t2)
# we close the connection to the database
conn.close()
# let's see some of the content
for k in range(5):
    print all_artist_names_sqlite[k][0]

In [ ]:
# now, let's find the artist that has the most songs in the dataset
# what we want to work with is artist ID, not artist names. Some artists
# have many names, usually because the song is "featuring someone else"
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id FROM songs"
res = conn.execute(q)
all_artist_ids = map(lambda x: x[0], res.fetchall())
conn.close()

# The Echo Nest artist id look like:
for k in range(4):
    print all_artist_ids[k]

In [65]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
cursor = conn.execute('select * from songs')
names = list(map(lambda x: x[0], cursor.description))
names

['track_id',
 'title',
 'song_id',
 'release',
 'artist_id',
 'artist_mbid',
 'artist_name',
 'duration',
 'artist_familiarity',
 'artist_hotttnesss',
 'year']

In [ ]:
# let's count the songs from each of these artists.
# We will do it first by iterating over the dataset.
# we prepare a dictionary to count files
files_per_artist = {}
for aid in all_artist_ids:
    files_per_artist[aid] = 0

# we prepare the function to check artist id in each file
def func_to_count_artist_id(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_id = GETTERS.get_artist_id(h5)
    files_per_artist[artist_id] += 1
    h5.close()

# we apply this function to all files
apply_to_all_files(msd_subset_data_path,func=func_to_count_artist_id)

# the most popular artist (with the most songs) is:
most_pop_aid = sorted(files_per_artist,
                      key=files_per_artist.__getitem__,
                      reverse=True)[0]
print most_pop_aid,'has',files_per_artist[most_pop_aid],'songs.'

In [ ]:
# of course, it is more fun to have the name(s) of this artist
# let's get it using SQLite
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_name FROM songs"
q += " WHERE artist_id='"+most_pop_aid+"'"
res = conn.execute(q)
pop_artist_names = map(lambda x: x[0], res.fetchall())
conn.close()
print 'SQL query:',q
print 'name(s) of the most popular artist:',pop_artist_names

# let's redo all this work in SQLite in a few seconds
t1 = time.time()
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id,artist_name,Count(track_id) FROM songs"
q += " GROUP BY artist_id"
res = conn.execute(q)
pop_artists = res.fetchall()
conn.close()
t2 = time.time()
print 'found most popular artist in',strtimedelta(t1,t2)
print sorted(pop_artists,key=lambda x:x[2],reverse=True)

In [ ]:
h5 = GETTERS.open_h5_file_read('data/MillionSongSubset/AdditionalFiles/subset_msd_summary_file.h5')

In [51]:
idxs = h5.root.metadata.songs.getWhereList('artist_name=="Radiohead"')
idxs

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: getWhereList() is pending deprecation, use get_where_list() instead. You may use the pt2to3 tool to update your source code.
  if __name__ == '__main__':


array([ 199, 1282, 1808, 2286, 5882, 6593, 6812, 8275, 8514, 8824, 9709])

In [71]:
for energy in h5.root.analysis.songs.cols.danceability:
    if energy > 0.0:
        print energy

In [72]:
filter(lambda x: x[:3] == 'get',GETTERS.__dict__.keys())

['get_danceability',
 'get_song_id',
 'get_release',
 'get_artist_hotttnesss',
 'get_title',
 'get_segments_timbre',
 'get_artist_longitude',
 'get_beats_confidence',
 'get_end_of_fade_in',
 'get_time_signature',
 'get_artist_id',
 'get_sections_start',
 'get_mode',
 'get_loudness',
 'get_artist_7digitalid',
 'get_artist_terms_freq',
 'get_similar_artists',
 'get_artist_terms_weight',
 'get_mode_confidence',
 'get_segments_loudness_max_time',
 'get_artist_familiarity',
 'get_song_hotttnesss',
 'get_time_signature_confidence',
 'get_artist_name',
 'get_key',
 'get_artist_playmeid',
 'get_artist_mbtags',
 'get_analysis_sample_rate',
 'get_year',
 'get_key_confidence',
 'get_artist_location',
 'get_tatums_start',
 'get_audio_md5',
 'get_bars_start',
 'get_bars_confidence',
 'get_artist_mbid',
 'get_track_7digitalid',
 'get_artist_terms',
 'get_segments_pitches',
 'get_segments_confidence',
 'get_segments_loudness_start',
 'get_energy',
 'get_segments_start',
 'get_segments_loudness_max',


In [93]:
h5 = GETTERS.open_h5_file_read('data/MillionSongSubset/data/B/C/B/TRBCBAM128F92E3044.h5')

In [108]:
# What metrics can we use?
h5 = GETTERS.open_h5_file_read('data/MillionSongSubset/data/B/C/B/TRBCBAM128F92E3044.h5')

# We have tempo 
print "tempo: {}".format(GETTERS.get_tempo(h5))

# We have loudness
print "loudness: {}".format(GETTERS.get_loudness(h5))

# We have mode
print "mode: {}".format(GETTERS.get_mode(h5))

tempo: 148.363
loudness: -4.935
mode: 1
